#Predicting sentiment from product reviews

#Fire up GraphLab Create

In [1]:
import graphlab

#Read some product review data

Loading reviews for a set of baby products. 

In [3]:
products = graphlab.SFrame('amazon_baby.gl/')

[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: /tmp/graphlab_server_1530176019.log


This non-commercial license of GraphLab Create for academic use is assigned to sjuyal.23@gmail.com and will expire on June 24, 2019.


#Let's explore this data together

Data includes the product name, the review text and the rating of the review. 

In [4]:
products.head()

name,review,rating
Planetwise Flannel Wipes,"These flannel wipes areOK, but in my opinion ...",3.0
Planetwise Wipe Pouch,it came early and was notdisappointed. i love ...,5.0
Annas Dream Full Quiltwith 2 Shams ...,Very soft and comfortableand warmer than it ...,5.0
Stop Pacifier Suckingwithout tears with ...,This is a product wellworth the purchase. I ...,5.0
Stop Pacifier Suckingwithout tears with ...,All of my kids have criednon-stop when I tried to ...,5.0
Stop Pacifier Suckingwithout tears with ...,"When the Binky Fairy cameto our house, we didn't ...",5.0
A Tale of Baby's Dayswith Peter Rabbit ...,"Lovely book, it's boundtightly so you may no ...",4.0
"Baby Tracker&reg; - DailyChildcare Journal, ...",Perfect for new parents.We were able to keep ...,5.0
"Baby Tracker&reg; - DailyChildcare Journal, ...",A friend of mine pinnedthis product on Pinte ...,5.0
"Baby Tracker&reg; - DailyChildcare Journal, ...",This has been an easy wayfor my nanny to record ...,4.0


#Build the word count vector for each review

In [7]:
products['word_count'] = graphlab.text_analytics.count_words(products['review'])

In [42]:
graphlab.text_analytics.stopwords()

{'a',
 'able',
 'about',
 'above',
 'according',
 'accordingly',
 'across',
 'actually',
 'after',
 'afterwards',
 'again',
 'against',
 'all',
 'allow',
 'allows',
 'almost',
 'alone',
 'along',
 'already',
 'also',
 'although',
 'always',
 'am',
 'among',
 'amongst',
 'an',
 'and',
 'another',
 'any',
 'anybody',
 'anyhow',
 'anyone',
 'anything',
 'anyway',
 'anyways',
 'anywhere',
 'apart',
 'appear',
 'appreciate',
 'appropriate',
 'are',
 'around',
 'as',
 'aside',
 'ask',
 'asking',
 'associated',
 'at',
 'available',
 'away',
 'awfully',
 'b',
 'be',
 'became',
 'because',
 'become',
 'becomes',
 'becoming',
 'been',
 'before',
 'beforehand',
 'behind',
 'being',
 'believe',
 'below',
 'beside',
 'besides',
 'best',
 'better',
 'between',
 'beyond',
 'both',
 'brief',
 'but',
 'by',
 'c',
 'came',
 'can',
 'cannot',
 'cant',
 'cause',
 'causes',
 'certain',
 'certainly',
 'changes',
 'clearly',
 'co',
 'com',
 'come',
 'comes',
 'concerning',
 'consequently',
 'consider',
 'con

In [17]:
products['tfidf'] = graphlab.text_analytics.tf_idf(products['word_count'])

In [19]:
products['tfidf'][0]

{'6': 3.3517201426807555,
 '8': 4.011816579364918,
 'a': 0.7401445560197378,
 'about': 1.9662039311604755,
 'also': 2.054362413958312,
 'and': 1.0220926250782232,
 'are': 3.682111140901507,
 'because': 1.876436094101983,
 'been': 2.3898149360112195,
 'blue-12': 12.120138869538158,
 'boyfor': 12.120138869538158,
 'but': 0.9086834287960969,
 'cloth': 8.023633158729837,
 'countwhich': 12.120138869538158,
 'fab': 8.335949235619898,
 'faces': 6.682059560614963,
 'flannel': 6.578875324379732,
 'for': 1.0177247964419551,
 'get': 1.7767847821765337,
 'had': 3.260620281519789,
 'handles.': 6.510667074353199,
 'hands': 3.8570640337355613,
 'have': 1.9984389859838765,
 'higher': 4.971793125638091,
 'i': 0.7076090091168256,
 'in': 0.7364227222577461,
 'issues': 4.378170969717473,
 'just': 1.4758098105794235,
 'keeping.': 9.555189512076621,
 'larger,': 7.237336946951788,
 'longer': 3.8203556975883703,
 'months': 2.413579026185347,
 'my': 0.5604634153789823,
 'need': 2.7232359465107665,
 'nicer,': 8

In [22]:
graphlab.canvas.set_target('browser')

In [23]:
products['name'].show()

Canvas is accessible via web browser at the URL: http://localhost:61602/index.html
Opening Canvas in default web browser.


#Examining the reviews for most-sold product:  'Vulli Sophie the Giraffe Teether'

In [24]:
giraffe_reviews = products[products['name'] == 'Vulli Sophie the Giraffe Teether']

In [25]:
len(giraffe_reviews)

785

In [26]:
giraffe_reviews['rating'].show(view='Categorical')

Canvas is updated and available in a tab in the default browser.


#Build a sentiment classifier

In [11]:
products['rating'].show(view='Categorical')

##Define what's a positive and a negative sentiment

We will ignore all reviews with rating = 3, since they tend to have a neutral sentiment.  Reviews with a rating of 4 or higher will be considered positive, while the ones with rating of 2 or lower will have a negative sentiment.   

In [27]:
#ignore all 3* reviews
products = products[products['rating'] != 3]

In [28]:
#positive sentiment = 4* or 5* reviews
products['sentiment'] = products['rating'] >=4

In [31]:
products.head(4)

Columns:
	name	str
	review	str
	rating	float
	word_count	dict
	tfidf	dict
	sentiment	int

Rows: 4

Data:
+-------------------------------+-------------------------------+--------+
|              name             |             review            | rating |
+-------------------------------+-------------------------------+--------+
|     Planetwise Wipe Pouch     | it came early and was not ... |  5.0   |
| Annas Dream Full Quilt wit... | Very soft and comfortable ... |  5.0   |
| Stop Pacifier Sucking with... | This is a product well wor... |  5.0   |
| Stop Pacifier Sucking with... | All of my kids have cried ... |  5.0   |
+-------------------------------+-------------------------------+--------+
+-------------------------------+-------------------------------+-----------+
|           word_count          |             tfidf             | sentiment |
+-------------------------------+-------------------------------+-----------+
| {'and': 3, 'love': 1, 'it'... | {'and': 0.6132555750469338... |     1     |
| {'and': 2, 'quilt': 1, 'it... | {'and': 0.4088370500312892... |     1     |
| {'ingenious': 1, 'and': 3,... | {'ingenious': 8.0596958589... |     1     |
| {'and': 2, 'parents!!': 1,... | {'and': 0.4088370500312892... |     1     |
+-------------------------------+-------------------------------+-----------+
[4 rows x 6 columns]

##Let's train the sentiment classifier

In [32]:
train_data,test_data = products.random_split(.8, seed=0)

In [34]:
sentiment_model = graphlab.logistic_classifier.create(train_data,
                                                     target='sentiment',
                                                     features=['word_count'],
                                                     validation_set=test_data)

WARNING: The number of feature dimensions in this problem is very large in comparison with the number of examples. Unless an appropriate regularization value is set, this model may not provide accurate predictions for a validation/test set.

Logistic regression:

--------------------------------------------------------

Number of examples          : 133448

Number of classes           : 2

...

| 6         | 13       | 3.000000  | 4.519288     | 0.899991          | 0.825967            |

| 10        | 18       | 1.000000  | 6.225025     | 0.988715          | 0.916256            |

+-----------+----------+-----------+--------------+-------------------+---------------------+

TERMINATED: Iteration limit reached.

This model may not be optimal. To improve it, consider increasing `max_iterations`.

#Evaluate the sentiment model

In [36]:
sentiment_model.evaluate(test_data, metric='roc_curve')

{'roc_curve': Columns:
 	threshold	float
 	fpr	float
 	tpr	float
 	p	int
 	n	int
 
 Rows: 100001
 
 Data:
 +-----------+----------------+----------------+-------+------+
 | threshold |      fpr       |      tpr       |   p   |  n   |
 +-----------+----------------+----------------+-------+------+
 |    0.0    |      1.0       |      1.0       | 27976 | 5328 |
 |   1e-05   | 0.909346846847 | 0.998856162425 | 27976 | 5328 |
 |   2e-05   | 0.896021021021 | 0.998748927652 | 27976 | 5328 |
 |   3e-05   | 0.886448948949 | 0.998462968259 | 27976 | 5328 |
 |   4e-05   | 0.879692192192 | 0.998284243637 | 27976 | 5328 |
 |   5e-05   | 0.875187687688 | 0.998212753789 | 27976 | 5328 |
 |   6e-05   | 0.872184684685 | 0.998177008865 | 27976 | 5328 |
 |   7e-05   | 0.868618618619 | 0.998034029168 | 27976 | 5328 |
 |   8e-05   | 0.864677177177 | 0.997998284244 | 27976 | 5328 |
 |   9e-05   | 0.860735735736 | 0.997962539319 | 27976 | 5328 |
 +-----------+----------------+----------------+-------+------

In [37]:
sentiment_model.show(view='Evaluation')

Canvas is updated and available in a tab in the default browser.


#Applying the learned model to understand sentiment for Giraffe

In [38]:
giraffe_reviews['predicted_sentiment'] = sentiment_model.predict(giraffe_reviews, output_type='probability')

In [21]:
giraffe_reviews.head()

name,review,rating,word_count,predicted_sentiment
Vulli Sophie the GiraffeTeether ...,He likes chewing on allthe parts especially the ...,5.0,"{'and': 1, 'all': 1,'because': 1, 'it': 1, ...",0.999513023521
Vulli Sophie the GiraffeTeether ...,My son loves this toy andfits great in the diaper ...,5.0,"{'and': 1, 'right': 1,'help': 1, 'just': 1, ...",0.999320678306
Vulli Sophie the GiraffeTeether ...,There really should be alarge warning on the ...,1.0,"{'and': 2, 'all': 1,'latex.': 1, 'being': 1, ...",0.013558811687
Vulli Sophie the GiraffeTeether ...,All the moms in my moms'group got Sophie for ...,5.0,"{'and': 2, 'one!': 1,'all': 1, 'love': 1, ...",0.995769474148
Vulli Sophie the GiraffeTeether ...,I was a little skepticalon whether Sophie was ...,5.0,"{'and': 3, 'all': 1,'old': 1, 'her.': 1, ...",0.662374415673
Vulli Sophie the GiraffeTeether ...,I have been reading aboutSophie and was going ...,5.0,"{'and': 6, 'seven': 1,'already': 1, 'love': 1, ...",0.999997148186
Vulli Sophie the GiraffeTeether ...,My neice loves her sophieand has spent hours ...,5.0,"{'and': 4, 'drooling,':1, 'love': 1, 'her.': 1, ...",0.989190989536
Vulli Sophie the GiraffeTeether ...,What a friendly face!And those mesmerizing ...,5.0,"{'and': 3, 'chew': 1,""don't"": 1, 'is': 1, ...",0.999563518413
Vulli Sophie the GiraffeTeether ...,We got this just for myson to chew on instea ...,5.0,"{'chew': 2, 'because': 1,'just': 2, 'what': 1, ...",0.970160542725
Vulli Sophie the GiraffeTeether ...,"My baby seems to likethis toy, but I could ...",3.0,"{'and': 2, 'already': 1,'in': 1, 'some': 1, ' ...",0.195367644588


##Sort the reviews based on the predicted sentiment and explore

In [39]:
giraffe_reviews = giraffe_reviews.sort('predicted_sentiment', ascending=False)

In [41]:
giraffe_reviews.head(5)

Columns:
	name	str
	review	str
	rating	float
	word_count	dict
	tfidf	dict
	predicted_sentiment	float

Rows: 5

Data:
+-------------------------------+-------------------------------+--------+
|              name             |             review            | rating |
+-------------------------------+-------------------------------+--------+
| Vulli Sophie the Giraffe T... | Sophie, oh Sophie, your ti... |  5.0   |
| Vulli Sophie the Giraffe T... | I'm not sure why Sophie is... |  4.0   |
| Vulli Sophie the Giraffe T... | I'll be honest...I bought ... |  4.0   |
| Vulli Sophie the Giraffe T... | We got this little giraffe... |  5.0   |
| Vulli Sophie the Giraffe T... | As a mother of 16month old... |  5.0   |
+-------------------------------+-------------------------------+--------+
+-------------------------------+-------------------------------+---------------------+
|           word_count          |             tfidf             | predicted_sentiment |
+-------------------------------+-------------------------------+---------------------+
| {'giggles': 1, 'all': 1, "... | {'giggles': 7.689322070694... |         1.0         |
| {'adoring': 1, 'find': 1, ... | {'adoring': 11.02152658087... |    0.999999999703   |
| {'all': 2, 'discovered': 1... | {'all': 3.415214558636941,... |    0.999999999392   |
| {'all': 2, "don't": 1, '(l... | {'all': 3.415214558636941,... |    0.99999999919    |
| {'cute': 1, 'all': 1, 'rev... | {'cute': 3.134944051546858... |    0.999999998657   |
+-------------------------------+-------------------------------+---------------------+
[5 rows x 6 columns]

##Most positive reviews for the giraffe

In [25]:
giraffe_reviews[0]['review']

"Sophie, oh Sophie, your time has come. My granddaughter, Violet is 5 months old and starting to teeth. What joy little Sophie brings to Violet. Sophie is made of a very pliable rubber that is sturdy but not tough. It is quite easy for Violet to twist Sophie into unheard of positions to get Sophie into her mouth. The little nose and hooves fit perfectly into small mouths, and the drooling has purpose. The paint on Sophie is food quality.Sophie was born in 1961 in France. The maker had wondered why there was nothing available for babies and made Sophie from the finest rubber, phthalate-free on St Sophie's Day, thus the name was born. Since that time millions of Sophie's populate the world. She is soft and for babies little hands easy to grasp. Violet especially loves the bumpy head and horns of Sophie. Sophie has a long neck that easy to grasp and twist. She has lovely, sizable spots that attract Violet's attention. Sophie has happy little squeaks that bring squeals of delight from Viol

In [26]:
giraffe_reviews[1]['review']

"I'm not sure why Sophie is such a hit with the little ones, but my 7 month old baby girl is one of her adoring fans.  The rubber is softer and more pleasant to handle, and my daughter has enjoyed chewing on her legs and the nubs on her head even before she started teething.  She also loves the squeak that Sophie makes when you squeeze her.  Not sure what it is but if Sophie is amongst a pile of her other toys, my daughter will more often than not reach for Sophie.  And I have the peace of mind of knowing that only edible and safe paints and materials have been used to make Sophie, as opposed to Bright Starts and other baby toys made in China.  Now that the research is out on phthalates and other toxic substances in baby toys, I think it's more important than ever to find good quality toys that are also safe for our babies to handle and put in their mouths.  Sophie is a must-have for every new mom in my opinion.  Even if your kid is one of the few that can take or leave her, it's worth

##Show most negative reviews for giraffe

In [27]:
giraffe_reviews[-1]['review']

"My son (now 2.5) LOVED his Sophie, and I bought one for every baby shower I've gone to. Now, my daughter (6 months) just today nearly choked on it and I will never give it to her again. Had I not been within hearing range it could have been fatal. The strange sound she was making caught my attention and when I went to her and found the front curved leg shoved well down her throat and her face a purply/blue I panicked. I pulled it out and she vomited all over the carpet before screaming her head off. I can't believe how my opinion of this toy has changed from a must-have to a must-not-use. Please don't disregard any of the choking hazard comments, they are not over exaggerated!"

In [28]:
giraffe_reviews[-2]['review']

"This children's toy is nostalgic and very cute. However, there is a distinct rubber smell and a very odd taste, yes I tried it, that my baby did not enjoy. Also, if it is soiled it is extremely difficult to clean as the rubber is a kind of porus material and does not clean well. The final thing is the squeaking device inside which stopped working after the first couple of days. I returned this item feeling I had overpaid for a toy that was defective and did not meet my expectations. Please do not be swayed by the cute packaging and hype surounding it as I was. One more thing, I was given a full refund from Amazon without any problem."